In [57]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

from implementation import *
from utilities import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load and clean the data

In [45]:
y,x,ids=load_csv_data("train.csv")
y_test,x_test,ids_test=load_csv_data("test.csv")

In [46]:
# tej colonnes où y'a des -999

x[x==-999]=np.NaN
z=x[:,~np.isnan(x).any(axis=0)] #z is our matrix of observations with size (number of observations, number of features without -999)

#z=one_hot_jet_num(z,17)
#z=build_poly_with_one_hot(z,5,17,True,3)

z2=interaction_prodbis(z,0,False)
z1 = build_poly(z,9)
z3=build_poly(z2,6)
#z = np.c_[z1,z2]
z=np.c_[z1,z3]
#add_ones(z)
#np.column_stack((z, np.ones(z.shape[0])))
#np.column_stack((z, np.ones(z.shape[0])))
print(z.shape)

1 0
2 0
2 1
3 0
3 1
3 2
4 0
4 1
4 2
4 3
5 0
5 1
5 2
5 3
5 4
6 0
6 1
6 2
6 3
6 4
6 5
7 0
7 1
7 2
7 3
7 4
7 5
7 6
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
10 0
10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
11 0
11 1
11 2
11 3
11 4
11 5
11 6
11 7
11 8
11 9
11 10
12 0
12 1
12 2
12 3
12 4
12 5
12 6
12 7
12 8
12 9
12 10
12 11
13 0
13 1
13 2
13 3
13 4
13 5
13 6
13 7
13 8
13 9
13 10
13 11
13 12
14 0
14 1
14 2
14 3
14 4
14 5
14 6
14 7
14 8
14 9
14 10
14 11
14 12
14 13
15 0
15 1
15 2
15 3
15 4
15 5
15 6
15 7
15 8
15 9
15 10
15 11
15 12
15 13
15 14
16 0
16 1
16 2
16 3
16 4
16 5
16 6
16 7
16 8
16 9
16 10
16 11
16 12
16 13
16 14
16 15
17 0
17 1
17 2
17 3
17 4
17 5
17 6
17 7
17 8
17 9
17 10
17 11
17 12
17 13
17 14
17 15
17 16
18 0
18 1
18 2
18 3
18 4
18 5
18 6
18 7
18 8
18 9
18 10
18 11
18 12
18 13
18 14
18 15
18 16
18 17
(250000, 1097)


In [50]:
x_test[x_test==-999]=np.NaN
z_test=x_test[:,~np.isnan(x_test).any(axis=0)] #z is our matrix of observations with size (number of observations, number of features without -999)
z1_test = build_poly(z_test,9)
z2_test=interaction_prodbis(z_test,0,False)
z3_test=build_poly(z2_test,6)
z_test = np.c_[z1_test,z3_test]
#z_test=np.c_[z1_test,z3_test]

1 0
2 0
2 1
3 0
3 1
3 2
4 0
4 1
4 2
4 3
5 0
5 1
5 2
5 3
5 4
6 0
6 1
6 2
6 3
6 4
6 5
7 0
7 1
7 2
7 3
7 4
7 5
7 6
8 0
8 1
8 2
8 3
8 4
8 5
8 6
8 7
9 0
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
10 0
10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
11 0
11 1
11 2
11 3
11 4
11 5
11 6
11 7
11 8
11 9
11 10
12 0
12 1
12 2
12 3
12 4
12 5
12 6
12 7
12 8
12 9
12 10
12 11
13 0
13 1
13 2
13 3
13 4
13 5
13 6
13 7
13 8
13 9
13 10
13 11
13 12
14 0
14 1
14 2
14 3
14 4
14 5
14 6
14 7
14 8
14 9
14 10
14 11
14 12
14 13
15 0
15 1
15 2
15 3
15 4
15 5
15 6
15 7
15 8
15 9
15 10
15 11
15 12
15 13
15 14
16 0
16 1
16 2
16 3
16 4
16 5
16 6
16 7
16 8
16 9
16 10
16 11
16 12
16 13
16 14
16 15
17 0
17 1
17 2
17 3
17 4
17 5
17 6
17 7
17 8
17 9
17 10
17 11
17 12
17 13
17 14
17 15
17 16
18 0
18 1
18 2
18 3
18 4
18 5
18 6
18 7
18 8
18 9
18 10
18 11
18 12
18 13
18 14
18 15
18 16
18 17


In [29]:
# cross validation for 2 --> TO DO: FOR K

Strain1_z, Stest1_z, Strain1_y, Stest1_y = split_data(z,y,0.8)

# standardize the data (only centered here)

#Strain1_z = standardize(Strain1_z)
#Stest1_z = standardize(Stest1_z)
#Strain1_y = standardize(Strain1_y)
#Stest_y = standardize(Stest1_y)

# Calculate MSE for result from least squares 

In [47]:
beta1 = least_squares(Strain1_y, Strain1_z)      #beta from y = X^T*beta --> missing the column of ones'
#print(beta1)
res1 = definitive_res(Stest1_z.dot(beta1))
#print(res1)
len(Stest1_y[Stest1_y==res1])/len(Stest1_y)

0.62676

# Calculate MSE for results from least squares GD and SGD

In [ ]:
initial_w = np.zeros(Strain1_z.shape[1])    #w is an array of size = number of features (i.e. number of columns in z)
max_iters = 50
gamma = 0.7

# GD

In [ ]:
beta2 = least_squares_GD(Strain1_y, Strain1_z, initial_w, max_iters, gamma)      #beta from y = X*beta --> missing the column of ones'
res2 = definitive_res(Stest1_z.dot(beta2))
len(Stest1_y[Stest1_y==res2])/len(Stest1_y)

# SGD

In [ ]:
batch_size = 1

beta3 = least_squares_SGD(Strain1_y, Strain1_z, initial_w, batch_size, max_iters, gamma)      #beta from y = X*beta --> missing the column of ones'
res3 = definitive_res(Stest1_z.dot(beta3))
len(Stest1_y[Stest1_y==res3])/len(Stest1_y)

# Calculate other LS for Ridge Regression 

In [ ]:
lambdas = np.logspace(-5, 0, 15)

# bluid poly
tx_tr = build_poly(Strain1_z, 1)
tx_te = build_poly(Stest1_z, 1)

# ridge regression with different lambda
rmse_tr = []
rmse_te = []
for ind, lambda_ in enumerate(lambdas):
    # ridge regression
    weight = ridge_regression(Strain1_y, tx_tr, lambda_)
    rmse_tr.append(np.sqrt(2 * compute_mse(Strain1_y, tx_tr, weight)))
    rmse_te.append(np.sqrt(2 * compute_mse(Stest1_y, tx_te, weight)))
print(rmse_tr, rmse_te)
#print("last weight:",weight)

In [60]:
lmin=-10
lmax=0
n=10
y=Strain1_y
x=Strain1_z
y_te=Stest1_y
x_te=Stest1_z
ratio=[]
ratios=[]
weights=[]
lambdas = np.logspace(lmin, lmax, n)
for ind, lambda_ in enumerate(lambdas):
    weight = ridge_regression(y, x, lambda_)
    ratio.append(ratio_prediction_threshold(y_te,x_te,weight,-0.1))
    ratios.append(ratio_prediction_threshold(y_te,x_te,weight,0.1))
    weights.append(weight)
    print(ind)
indx1=np.argmax(ratio)
indx2=np.argmax(ratios)
weight=weights[indx]
lambdaa=lambdas[indx]


0
1
2
3
4
5
6
7
8
9


In [61]:
print(np.max(ratio),indx)
print(np.max(ratios),indx)

0.8043 4
0.8075 4


In [54]:
pred=definitive_res(z_test.dot(weight))
create_csv_submission(ids_test,pred,"testsubs")

ValueError: shapes (568238,1097) and (905,) not aligned: 1097 (dim 1) != 905 (dim 0)